In [0]:

jdbc_url = (
    "jdbc:sqlserver://sqldbdbxtraining.database.windows.net:1433;"
    "database=sqldb-nadbxtraining;"
    "encrypt=true;trustServerCertificate=false;"
    "hostNameInCertificate=*.database.windows.net;"
)
 
connection_props = {
    "user": "sqladministrator@sqldbdbxtraining",
    "password": dbutils.secrets.get("dbx-ss-kv-natraining-2", "sqladministrator-password"),
    "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}
 
print(f"Connection URL: {jdbc_url}")
print(f"Reading Table: {jdbcTable}")
df = spark.read.jdbc(
    url=jdbc_url, 
    table=jdbcTable, 
    properties=connection_props
)

# Display the data in an interactive Databricks table
display(df)

# Optional: Print the schema to verify data types
# df.printSchema()

In [0]:
from pyspark.sql.functions import current_timestamp

# 1. Read the data from SQL Server via JDBC
df = spark.read.jdbc(
    url=jdbc_url, 
    table=jdbcTable, 
    properties=connection_props
)

# 2. Add an ingestion timestamp
df_with_timestamp = df.withColumn("ingestion_timestamp", current_timestamp())

# 3. Create the table if it doesn't exist (using Iceberg format)
# Write an empty DataFrame with the correct schema to create the table
(df_with_timestamp.limit(0)
    .write
    .format("iceberg")
    .option("location", f"abfss://<your-container>@<your-storage-account>.dfs.core.windows.net/na-dbxtraining/biju_bronze/customer_iceberg")
    .saveAsTable("`na-dbxtraining`.biju_bronze.customer_iceberg")
)

# 4. Perform the Batch Append to the Iceberg table
(df_with_timestamp.writeTo("`na-dbxtraining`.biju_bronze.customer_iceberg")
    .append())

print("✓ Batch load from SQL Server to Iceberg table complete.")

In [0]:
%sql
select * from `na-dbxtraining`.biju_bronze.customer_iceberg